In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

Data folder is C:\Users\lfranceschi\DATASETS
Experiment save directory is  C:\Users\lfranceschi\EXPERIMENTS


In [4]:
far.utils._check()

3


In [16]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])

In [17]:
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

io_optim = far.MomentumOptimizer(far.get_hyperparameter('eta', 0.1), 
                                   far.get_hyperparameter('mu', .9))

io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)

In [18]:
rhg = far.ReverseHg()
rhg.compute_gradients(oo, io_optim_dict)

optim_oo = tf.train.AdamOptimizer(.01)
ts_hy = optim_oo.apply_gradients(rhg.hgrads_hvars())

In [19]:
tf.global_variables()

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable/Momentum:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Momentum:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda/Adam:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda/Adam_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta/Adam:0' shape=() dtype=float32_ref>,
 <tf.Variable 'eta/Adam_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu/Adam:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu/Adam_1:0' shape=() dtype=float32_ref>]

In [20]:
far.utils.hyperparameters()

[<tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

In [21]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [22]:
rhg.run(10)

In [23]:
ss.run(far.utils.hypergradients())

[array([[ 0.2233436 ,  1.34006166],
        [-0.2233436 ,  0.        ]], dtype=float32), 13.395198, 0.28557462]

In [24]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))

# check with rfho

In [25]:
import rfho as rf

Environment variable RFHO_EXP_FOLDER not found. Current directory will be used
Experiment save directory is  C:\Users\lfranceschi\PycharmProjects\FAR-HO\tests\Experiments
Environment variable RFHO_DATA_FOLDER not found. Variables HELP_WIN and HELP_UBUNTU contain info.
Data folder is C:\Users\lfranceschi\PycharmProjects\FAR-HO\tests


In [26]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=1)

In [27]:
dyn = rf.MomentumOptimizer.create(w, 0.1, 0.9, loss=c)

In [28]:
hyperg = rf.HyperOptimizer(dyn, {co: lmbd}, rf.ReverseHG)

In [29]:
hyperg.initialize()

True

In [30]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [32]:
print('rfho')
print(ss.run(hyperg.hyper_gradients.hyper_gradient_vars))
print('far')
ss.run(far.utils.hypergradients())  # there are also other hyperparameters

rfho
[array([[ 0.22334361,  1.34006166],
       [-0.22334361,  0.        ]], dtype=float32)]
far


[array([[ 0.2233436 ,  1.34006166],
        [-0.2233436 ,  0.        ]], dtype=float32), 13.395198, 0.28557462]

# END... RESULTS ARE (almost the) SAME